### This notebook explores if you really need a custom Agent tool for doing calculations or the LLM can handle the calculations directly.

As a part of exercise, I crawled the page as document which has fees information: https://www.canada.ca/en/immigration-refugees-citizenship/services/immigrate-canada/express-entry.html which contains information about application fees. Chunked the document into paragraphs, and asked questions like this:

User Query: I have 3 dependent children and a spouse. How much will be my total applications fees?

This exercise shows how the LLM can do basic maths operations and we may not need a custom tool for the same.


In [1]:
from llama_index.core import SimpleDirectoryReader

# Load your documents
documents = SimpleDirectoryReader(input_files=["../data/docs/immigration-refugees-citizenship_services_immigrate-canada_express-entry.txt"]).load_data()

### Chunk documents on custom splitter.

In [2]:
from llama_index.core.text_splitter import TokenTextSplitter

class ParagraphSplitter(TokenTextSplitter):
    def split_text(self, text):
        """
        Splits text on 3 new line (Text separated by 2 empty lines)

        """
        paragraphs = [p.strip() for p in text.split('\n\n\n') if p.strip()]
        return paragraphs

splitter = ParagraphSplitter()


### Example to check how the custom split works on raw text

In [3]:

text = """This is para1.


This is para2.

This is parA3"""

nodes = splitter.split_text(text)

for p in nodes:
    print(p)
    print ("-------------------")

This is para1.
-------------------
This is para2.

This is parA3
-------------------


### Split on documents

In [4]:

nodes = splitter.get_nodes_from_documents(documents)


# Apply splitter to each document
all_paragraphs = []
for doc in documents:
    nodes = splitter.split_text(doc.text)
    all_paragraphs.extend(nodes)

### Storing embeddings on Faiss vector store on paragraph split chunks


In [5]:
# Initialize embeddings, used for encoding documents into embedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

/home/nausheenfatma/.cache/pypoetry/virtualenvs/canadaimmigrationhelpdesk-usinggenai-s0qUKeX3-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex
import faiss


In [7]:
# 1. Load documents
#documents = SimpleDirectoryReader('path/to/docs').load_data()
from llama_index.core.text_splitter import SentenceSplitter
from llama_index.core.schema import TextNode

documents = SimpleDirectoryReader(input_files=["../data/docs/immigration-refugees-citizenship_services_immigrate-canada_express-entry.txt"]).load_data()

# 2. Split into paragraphs strings
splitter = ParagraphSplitter()
all_nodes = []

for doc in documents:
    paragraphs = splitter.split_text(doc.text)
    # Convert to Node objects
    nodes = [TextNode(text=p) for p in paragraphs]
    all_nodes.extend(nodes)


## 3. Set up embedding model
# embed_model = OpenAIEmbedding()  # or your own embedding model
embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")

# # 4. Create FAISS vector store and embed
# Create FAISS index
dimension = 384  # Dimension for MiniLM embeddings
faiss_index = faiss.IndexFlatL2(dimension)

# Construct the FaissVectorStore
vector_store = FaissVectorStore(faiss_index=faiss_index)


# Create a StorageContext to use with LlamaIndex
storage_context = StorageContext.from_defaults(vector_store=vector_store)


index = VectorStoreIndex(
        nodes, storage_context=storage_context, 
        embed_model=embed_model)

# 5. Persist FAISS index (optional)
# faiss.write_index(index.vector_store.faiss_index, "faiss_index_chunks.idx")

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [8]:
nodes

[TextNode(id_='d978bbf2-8028-4db2-a5e4-9ee6702c5084', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='This is content for https://www.canada.ca/en/immigration-refugees-citizenship/services/immigrate-canada/express-entry', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'),
 TextNode(id_='393374aa-16ce-426c-b729-a8d23db410d8', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text='You are here:\n - Canada.ca\n - Immigration and citizenship\n - Immigrate to Canada', mimetype='text/plain', start_char_idx=None, end_char_idx=None, metadata_seperator='\n', text_template='{metadata_str}\n\n{content}'),
 TextNode(id_='f9444559-3d08-4c2c-a90d-6babc3b5bfd3', embeddin

retriever = index.as_retriever()

In [9]:
retriever = index.as_retriever()

In [10]:
from llama_index.llms.ollama import Ollama
llm = Ollama(model="llama2", request_timeout=100.0)

# Query function
def query_rag_system(question):
    retrieved_docs = retriever.retrieve(question)
    context = "\n".join([doc.text for doc in retrieved_docs]) 
    prompt = f"Context:\n{context}\n\n Question: {question}\n\n\n\nAnswer:"
    response = llm.complete(prompt)
    return response

In [11]:
response = query_rag_system("I have 3 dependent children and a spouse. How much will be my total applications fees?")

In [12]:
response.text

'The total fee for your application, including your spouse and three dependent children, would be $CAN 1,525 + ($CAN 260 x 3) = $CAN 4,385.'

In [13]:
response = query_rag_system("I am applying for myself and my spouse. How much will be my total applications fees?")

In [14]:
response.text

'Based on the information provided in the context, if you are applying for yourself and your spouse, the total application fees would be $CAN 1,525 + $CAN 1,525 = $CAN 3,050.'

In [15]:
response = query_rag_system("I am a single mother with 1 child. How much is my application fees.")

In [16]:
response.text

'Based on the information provided in the context, your application fee as a single mother with one child would be $CAN 1,525. This is the same amount as the application fee for a spouse or common-law partner, as well as the fee for each dependent child. Therefore, your application fee would be $CAN 1,525.'

In [17]:
# above example failed. rewrite prompt 
response = query_rag_system("I am a single mother with 1 dependent child. How much is my application fees.")

In [18]:
response.text

'Based on the information provided in the context, the application fee for a single mother with one dependent child is $CAN 1,525. This is the same as the application fee for a family of three or more members.'

In [ ]:
# the above two answers gave wrong output, fix prompts.

response = query_rag_system("I am a mother applying for myself and 1 dependent child. How much is my total application fees.")
response.text

'Based on the information provided in the context, the total application fee for a mother applying for herself and one dependent child would be $CAN 1,525 + $CAN 260 = $CAN 1,785.'

The above exercise shows that building cutom tool for simple mathematical problems can largely be handled using the LLMs directly.

### When do we really need to build custom tools then?

You should consider building your own custom agent tool when:

- You need access to a specific API, database, or system
E.g., querying a proprietary dataset, scraping a niche website, or interacting with a custom backend service.

- You want to encapsulate complex business logic
If the logic is too domain-specific (e.g., insurance claim validation or NLP over legal documents), custom tools help abstract and reuse that logic.

- Built-in tools don't cover your use case
You might need to integrate with IoT devices, internal microservices, or specialized hardware.

- You need control over latency or caching
Some tasks may benefit from local caching, retry logic, or rate limiting—these can be handled inside a custom tool.



### Do Mathematical Operations Require Tools?
Simple math operations (like 2 + 2, or sqrt(16)) do not require tools—LLMs can usually handle them.

But you should use a tool when:

- You need precision math or long-form calculations (e.g., compound interest, probability calculations, matrix ops).

- You want to use NumPy, SymPy, or SciPy.

- You need math inside a reasoning chain (e.g., the result feeds into another decision).